<a href="https://colab.research.google.com/github/Vedang22MIC0017/GPU-Programming/blob/main/vectoraddition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y build-essential
!apt-get install -y cuda

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,772 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,160 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,152

In [9]:
import os
from IPython.utils import io
from IPython.core.magic import register_cell_magic
# magic function to let us write c code in python for cuda
@register_cell_magic
def cuda_compile_run(line, cell):
    # Save .cu code to temp file
    with open('temp.cu', 'w') as f:
        f.write(cell)

    # Compile with nvcc
    compile_result = os.system('/usr/local/cuda/bin/nvcc temp.cu -o temp_exe')
    if compile_result != 0:
        print("Compilation failed")
        return

    # Run compiled executable
    output = os.popen('./temp_exe').read()
    print(output)

In [8]:
%%cuda_compile_run

#include <stdio.h>

__global__ void vectorAdd(float *a, float *b, float *c, int n) { //global is for calling the gpu from the cpu
    int i = threadIdx.x + blockIdx.x * blockDim.x; //allocating the threads by their id
    if (i < n)
        c[i] = a[i] + b[i]; // performing addition
}

int main() {
    const int N = 10;
    size_t size = N * sizeof(float);

    float h_a[N], h_b[N], h_c[N];
    for (int i = 0; i < N; i++) {
        h_a[i] = float(i); // 0,1,2,3,4,5,6,7,8,9
        h_b[i] = float(i * 2); // 0,2,6,8,10...
    }

    float *d_a, *d_b, *d_c; //allocating memory for gpu
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice); //copying data from cpu to gpu
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    vectorAdd<<<1, N>>>(d_a, d_b, d_c, N); //calling function for 1 block with n =10 threads then passing formal parameters

    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);//copying the result from gpu to cpu now it is stored in cpu

    for (int i = 0; i < N; i++) {
        printf("%.1f + %.1f = %.1f\n", h_a[i], h_b[i], h_c[i]); //printing the result from cpu
    }

    cudaFree(d_a); //releasing the allocated gpu memory
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

0.0 + 0.0 = 0.0
1.0 + 2.0 = 0.0
2.0 + 4.0 = 0.0
3.0 + 6.0 = 0.0
4.0 + 8.0 = 0.0
5.0 + 10.0 = 0.0
6.0 + 12.0 = 0.0
7.0 + 14.0 = 0.0
8.0 + 16.0 = 0.0
9.0 + 18.0 = 0.0

